## 序列模型

### 建模过程

&emsp;&emsp;在时间$t$观察到一个数据$x_{t}$, 那么如果有$T$个时间的话，我们可以得到$T$个不独立的随机变量：$\left(x_{1}, \ldots x_{T}\right) \sim p(\mathbf{x})$。机器学习就是在求这个$p(\mathbf{x})$。

&emsp;&emsp;我们又已知条件概率公式，可以表示为：

$$
p(a, b)=p(a) p(b \mid a)=p(b) p(a \mid b)
$$

&emsp;&emsp;有了上述的条件概率公式的话，我们就可以把$p(\mathbf{x})$表示出来：

$$
p(\mathbf{x})=p\left(x_{1}\right) \cdot p\left(x_{2} \mid x_{1}\right) \cdot p\left(x_{3} \mid x_{1}, x_{2}\right) \cdot \ldots p\left(x_{T} \mid x_{1}, \ldots x_{T-1}\right)
$$

&emsp;&emsp;也很好理解，比如想要算$t$时刻发生的事情，我们就需要去知道$t$时刻之前发生了什么事情。

&emsp;&emsp;当然我们也可以逆向推出：

$$
p(\mathbf{x})=p\left(x_{T}\right) \cdot p\left(x_{T-1} \mid x_{T}\right) \cdot p\left(x_{T-2} \mid x_{T-1}, x_{T}\right) \cdot \ldots p\left(x_{1} \mid x_{2}, \ldots x_{T}\right)
$$

&emsp;&emsp;反序在某些时候是有意义的，比如我已经知道未来的事情了，我们来推之前的事情会是什么样子的。但是物理上不一定可行。

### 序列模型核心

&emsp;&emsp;序列模型的核心就是要计算：给定$t$时刻之前的数据，我们来计算$t$时刻事情发生的概率：

$$
p\left(x_{t} \mid x_{1}, \ldots x_{t-1}\right)=p\left(x_{t} \mid f\left(x_{1}, \ldots x_{t-1}\right)\right)
$$

&emsp;&emsp;上述的$f$函数就可以理解为给定过去的$t-1$个数据来训练一个模型。上述这种对见过的数据建模也称为**自回归模型**。

&emsp;&emsp;核心的事情就是如何来计算$f\left(x_{1}, \ldots x_{t-1}\right)$。第二个就是给定$f$之后怎么来计算这个$p$。

### 方案一：马尔可夫假设

&emsp;&emsp;第一个方案就是马尔可夫假设：假设当前数据值跟$\tau$个过去数据点相关。当我们预测新的数据的时候，我们就只看过去的$\tau$个数据。那么概率建模就可以表示成：

$$
p\left(x_{t} \mid x_{1}, \ldots x_{t-1}\right)=p\left(x_{t} \mid x_{t-\tau}, \ldots x_{t-1}\right)=p\left(x_{t} \mid f\left(x_{t-\tau}, \ldots x_{t-1}\right)\right)
$$

&emsp;&emsp;这样我们训练$f$的时候，我们训练一个MLP就可以了，这就可以变成最简单的回归问题。

### 方案二：潜变量模型

&emsp;&emsp;另外的一种方案就是引入一个潜变量$h_{t}$来表示过去的信息$h_{t}=f\left(x_{1}, \ldots x_{t-1}\right)$。和我们的隐变量从统计意义上来说有一点点区别，从概念上来说更加广一点，可以认为是隐变量稍微推广了一点的类型。

&emsp;&emsp;这样的话，我们有：$x_{t}=p\left(x_{t} \mid h_{t}\right)$。

&emsp;&emsp;这里我们就会有两个模型

1. 模型1:依据潜变量和输入x来计算新的潜变量。
2. 模型2: 给定新的潜变量和前一个时刻的输入x，怎么来计算新的x。

&emsp;&emsp;这样的话，就将其拆分成了两个模型，每个模型只和一个或者两个变量相关，相对来说会比较容易一点。

&emsp;&emsp;**潜变量模型就是使用潜变量来概括历史信息。** 隐变量(hidden)一般是真实存在的东西，只不过我们没有观察到。但是潜变量(laten)包括了隐变量这一个假设，他可以是真实不存在的东西，你根本就观察不到。

1. 潜变量模型和隐马尔可夫模型有什么区别？ 潜变量模型是可以使用隐马尔可夫假设的。



## 语言模型

&emsp;&emsp;语言模型说的是给定一个文本序列$x_{1}, \cdots, x_{T}$, 语言模型的目标是估计联合概率$p(x_{1},\cdots, x_{T})$。相关的应用有：

1. 做个预训练模型，像`BERT`、`GPT-3`这样的。

2. 生成文本，比如说给定前面的几个词，不断的使用$x_{t} \sim p(x_{t} | x_{0}, \sim x_{t-1})$生成后续文本。

3. 判断多个序列中哪个更常见，比如语音模型输出多个可选项，因为一个意思可以用多种话来说，因此判断哪种方式出现的概率比较高也是有必要的。

### 使用计数来建模

&emsp;&emsp;假设序列长度为`2`，我们想要去预测$p(x_{1}, x_{2})$, 假设我们总共有$n$个词，$n(x_{1}), n(x_{1}, x_{2})$是单个词和连续词对出现的概率：

$$
p(x_{1}, x_{2}) = p(x_{1}) p(x_{1} | x_{2}) = \frac{n(x_{1})}{n} \frac{n(x_{1}, x_{2})}{n(x_{2})}
$$

#### N元语法

&emsp;&emsp;当序列很长时，因为文本量不够大，很可能$n(x_{1}, \cdots, x_{T}) \leq 1$。一旦说会出现0个的现象的话，整个概率乘起来就会等于`0`了。可以采用马尔可夫假设来缓解这个问题：

#### 一元语法 

$$
p(x_{1}, x_{2}, x_{3}, x_{4}) = p(x_{1}) p(x_{2}) p(x_{3}) p(x_{4}) = \frac{n(x_{1})}{n} \frac{n(x_{2})}{n} \frac{n(x_{3})}{n} \frac{n(x_{4})}{n}
$$

#### 二元语法

$$
p(x_{1}, x_{2}, x_{3}, x_{4}) = p(x_{1}) p(x_{2}｜x_{1}) p(x_{3}|x_{2}) p(x_{4} | x_{3}) = \frac{n(x_{1})}{n} \frac{n(x_{1}, x_{2})}{n(x_{1})} \frac{n(x_{2}, x_{3})}{n(x_{2})} \frac{n(x_{3}, x_{4})}{n(x_{4})}
$$

## 门控循环单元 GRU

&emsp;&emsp;在`RNN`中，所有隐藏信息都放在一个隐藏状态里面，当时间步很长的时候，里面就会累计了太多的东西。之前的信息就不是那么好抽取出来。并且对于一个输入序列，每个观察值都不是说是同等重要的。

1. **更新门**：能关注的机制（更新门）。$Z_{t}$

2. **重置门**：能遗忘的机制（重置门）。$R_{t}$

$$
Z_{t} = \sigma(X_{t} W_{xz} + H_{t-1} W_{hz} + b_{z}) \\
R_{t} = \sigma(X_{t} W_{xr} + H_{t-1} W_{hr} + b_{r})
$$

3. **候选隐藏状态**:

&emsp;&emsp;之后基于这两个门的输出，我们来计算候选隐藏状态:

$$
\tilde{\boldsymbol{H}}_{t}=\tanh \left(\boldsymbol{X}_{t} \boldsymbol{W}_{x h}+\left(\boldsymbol{R}_{t} \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{h h}+\boldsymbol{b}_{h}\right)
$$

&emsp;&emsp;如果将其与`RNN`做对比的话，我们可以发现，如果不看$R_{t}$的话，他就和之前的`RNN`是一样的。

4. **隐状态**:

$$
\boldsymbol{H}_{t}=\mathbf{Z}_{t} \odot \boldsymbol{H}_{t-1}+\left(1-\boldsymbol{Z}_{t}\right) \odot \tilde{\boldsymbol{H}}_{t}
$$

## 长短期记忆网络 LSTM

1. **忘记门**: 将值朝0减少。$F_{t}$

2. **输入门**: 决定不是忽略掉输入数据。$I_{t}$

3. **输出门**: 决定是不是使用隐状态。$O_{t}$

$$
\begin{aligned}
\boldsymbol{I}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x i}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h i}+\boldsymbol{b}_{i}\right) \\
\boldsymbol{F}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x f}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h f}+\boldsymbol{b}_{f}\right) \\
\boldsymbol{O}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x o}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h o}+\boldsymbol{b}_{o}\right)
\end{aligned}
$$

4. **候选记忆单元**：

$$
\tilde{\boldsymbol{C}}_{t}=\tanh \left(\boldsymbol{X}_{t} \boldsymbol{W}_{x c}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h c}+\boldsymbol{b}_{c}\right)
$$

5. **记忆单元**:

$$
\boldsymbol{C}_{t}=\boldsymbol{F}_{t} \odot \boldsymbol{C}_{t-1}+\boldsymbol{I}_{t} \odot \tilde{\boldsymbol{C}}_{t}
$$

&emsp;&emsp;`LSTM`与`RNN、GRU`的区别是里面所含的状态有两个，一个是$C$，另外一个是$H$。在`GRU`中，认为当前的信息但愿与之前的全局信息是此消彼长的关系，所以直接用$1 - z_{t}$替换$I_{t}$了, 简单粗暴。

6. **隐状态**:

$$
\boldsymbol{H}_{t}=\boldsymbol{O}_{t} \odot \tanh \left(\boldsymbol{C}_{t}\right)
$$

&emsp;&emsp;$\tanh$是为了保证值是在`+1`到`-1`之间的。

## 模型类别

从模型上分类，循环神经网络有单向循环，双向循环，和多层单向或双向循环。

## 衡量指标

- 困惑度(perplexity)

&emsp;&emsp;衡量一个语言模型的好坏可以用平均交叉熵来衡量：

$$
\pi = \frac{1}{n} \sum_{i=1}^{n} -log p(x_{t} | x_{t-1}, \cdots)
$$

&emsp;&emsp;$p$是语言模型的预测概率，$x_{t}$是真实词。但是由于一些历史原因`NLP`中采用困惑度$exp(\pi)$来衡量, 这样的话1表示完美，无穷大是最差的情况。

## 梯度裁剪

&emsp;&emsp;迭代中计算这$T$个时间步上的梯度，在反向传播过程中产生长度为$O(T)$的矩阵乘法链，导致数值不稳定。梯度裁剪能有效预防梯度爆炸。梯度裁剪说的是如果梯度长度超过$\theta$，那么拖影回长度$\theta$。

$$
\mathbf{g} \leftarrow \min \left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}
$$

&emsp;&emsp;因为时间步为$T$，所以会等价于$T$个$MLP$。所以他会容易产生梯度爆炸。

## 小结

1. `RNN`中的梯度消失/梯度爆炸和普通的`MLP`或者深层`CNN`中梯度消失/剃度爆炸的含义不一样。`MLP/CNN`中不同的层有不同的参数，各是各的梯度；而`RNN`中同样的权重在各个时间步共享，最终的梯度 `g = 各个时间步的梯度 g_t` 的和。

2. 由1中所述，**RNN中总的梯度是不会消失的**。即便梯度越传越弱，那也只是远距离的梯度消失, 但是近距离的梯度并不会消失，所有梯度之和便不会消失。`RNN`所谓梯度消失的真正含义是: **梯度被近距离梯度主导，导致模型难以学到远距离的依赖关系**。

3. **`LSTM`中梯度的传播有很多条路径**, $c_{t-1} \rightarrow c_{t}=f_{t} \odot c_{t-1}+i_{t} \odot \hat{c_{t}}$这条路径上只有逐元素相乘和相加的操作，梯度流最稳定；但是其它路径(例如$c_{t-1} \rightarrow h_{t-1} \rightarrow i_{t} \rightarrow c$)上梯度流与普通`RNN`类似，照样会发生相同的权重矩阵反复连乘。

4. `LSTM`刚提出来的时候没有遗忘门，或者说相当于$f_{t} = 1$, 这时候在$c_{t-1} \rightarrow c_{t}$直接相连的短路径上，$\frac{dl}{dc_{t}}$可以无损地传递给$\frac{dl}{dc_{t-1}}$, 从而这条路径熵的梯度畅通无阻，不会消失，类似于`ResNet`中的残差连接。

5. 但是在**其他路径**上，`LSTM`的梯度流和普通`RNN`没有太大区别，依然会爆炸或者消失。由于总的远距离梯度 = 各条路径的远距离梯度之和，即便其他远距离路径梯度消失了，只要保证有一条远距离路径(就是上面说的那条高速公路) 梯度不消失，总的远距离梯度就不会消失 (正常梯度 + 消失梯度 = 正常梯度)。因此`LSTM`通过改善**一条路径上**的梯度问题拯救了**总体的远距离梯度**。

6. 同样，因为总的远距离梯度 = 各条路径的远距离梯度之和，高速公路上梯度流比较稳定，但其他路径上梯度有可能爆炸，此时总的远距离梯度 = 正常梯度 + 爆炸梯度 = 爆炸梯度，**因此LSTM仍然有可能发生爆炸**。不过，由于LSTM的其他路径非常崎岖， 和普通RNN相比，多经过了很多次激活函数（导数都小于1），因此**LSTM发生梯度爆炸的频率要低得多**。实践中，梯度爆炸一般通过梯度裁剪来解决。

7. 对于现在常用的带遗忘门的`LSTM`来说，`6`中的分析依然成立，而`5`分为两种情况：其一是遗忘门接近`1`（例如模型初始化时会把`forget bias`设置成较大的正数，让遗忘门饱和），这时候远距离梯度不小时；其二是遗忘门接近`0`，但这时模型是故意阻断梯度流的，这不是`bug`而是`feature`（例如情感分析任务中有一条样本 “A，但是 B”，模型读到“但是”后选择把遗忘门设置成 0，遗忘掉内容 A，这是合理的）。当然，常常也存在f介于[0, 1]之间的情况，在这种情况下只能说LSTM改善（而非解决）了梯度消失的状况。

- [Written Memories: Understanding, Deriving and Extending the LSTM](https://r2rt.com/written-memories-understanding-deriving-and-extending-the-lstm.html)

- [Why LSTMs Stop Your Gradients From Vanishing: A View from the Backwards Pass](https://weberna.github.io/blog/2017/11/15/LSTM-Vanishing-Gradients.html)

## 参考

- [LSTM如何来避免梯度弥散和梯度爆炸？](https://www.zhihu.com/question/34878706/answer/665429718)
- [RNN梯度消失/爆炸的深度好文](https://kexue.fm/archives/7888)
- https://github.com/huseinzol05/Machine-Learning-Numpy